# Imports

In [1]:
%pdb off
# %load_ext viztracer
%load_ext autoreload
%autoreload 2
import sys
import importlib
from pathlib import Path

import numpy as np

# required to enable non-blocking interaction:
# %gui qt
# !env QT_API="pyqt5"
%gui qt5
# %gui qt6
# from PyQt5.Qt import QApplication
# # start qt event loop
# _instance = QApplication.instance()
# if not _instance:
#     _instance = QApplication([])
# app = _instance

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
from benedict import benedict # https://github.com/fabiocaccamo/python-benedict#usage

# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext

from PendingNotebookCode import _perform_batch_plot, _build_batch_plot_kwargs, find_local_session_paths, batch_load_session, batch_programmatic_figures, SessionBatchProgress, batch_extended_programmatic_figures

session_batch_status = {}
enable_saving_to_disk = False

global_data_root_parent_path = Path(r'W:\Data') # Windows Apogee
# global_data_root_parent_path = Path(r'/media/MAX/Data') # Diba Lab Workstation Linux
# global_data_root_parent_path = Path(r'/Volumes/MoverNew/data') # rMBP
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF


C:\Users\pho\miniconda3\envs\phoviz_ultimate\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log


# Load Pipeline

In [2]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'

## Data must be pre-processed using the MATLAB script located here: 
#     neuropy/data_session_pre_processing_scripts/KDIBA/IIDataMat_Export_ToPython_2022_08_01.m
# From pre-computed .mat files:

local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=self.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

## Animal `gor01`:
local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='gor01', exper_name='one') # IdentifyingContext<('kdiba', 'gor01', 'one')>
local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name) # 'gor01', 'one'
local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=['PhoHelpers', 'Spike3D-Minimal-Test', 'Unused'])

# local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='gor01', exper_name='two')
# local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name)
# local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=[])

### Animal `vvp01`:
# local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='vvp01', exper_name='one')
# local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name)
# local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=[])

# local_session_parent_context = local_session_root_parent_context.adding_context(collision_prefix='animal', animal='vvp01', exper_name='two')
# local_session_parent_path = local_session_root_parent_path.joinpath(local_session_parent_context.animal, local_session_parent_context.exper_name)
# local_session_paths_list, local_session_names_list =  find_local_session_paths(local_session_parent_path, blacklist=[])

## Build session contexts list:
local_session_contexts_list = [local_session_parent_context.adding_context(collision_prefix='sess', session_name=a_name) for a_name in local_session_names_list] # [IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-07_11-26-53')>, ..., IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-13_14-42-6')>]

## Initialize `session_batch_status` with the NOT_STARTED status if it doesn't already have a different status
for curr_session_basedir in local_session_paths_list:
    curr_session_status = session_batch_status.get(curr_session_basedir, None)
    if curr_session_status is None:
        session_batch_status[curr_session_basedir] = SessionBatchProgress.NOT_STARTED # set to not started if not present
        # session_batch_status[curr_session_basedir] = SessionBatchProgress.COMPLETED # set to not started if not present
        

session_batch_status

local_session_names_list: ['2006-6-07_11-26-53', '2006-6-08_14-26-15', '2006-6-09_1-22-43', '2006-6-09_3-23-37', '2006-6-12_15-55-31', '2006-6-13_14-42-6']


{WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-07_11-26-53'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-09_1-22-43'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-09_3-23-37'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-12_15-55-31'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>,
 WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-13_14-42-6'): <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>}

In [ ]:
## Run batch queue
for curr_sess_ctx, curr_session_basedir, curr_session_name in zip(local_session_contexts_list, local_session_paths_list, local_session_names_list):
    print(f'curr_session_basedir: {str(curr_session_basedir)}')
    curr_session_status = session_batch_status.get(curr_session_basedir, None)
    if curr_session_status.name != SessionBatchProgress.COMPLETED.name:
        session_batch_status[curr_session_basedir] = SessionBatchProgress.NOT_STARTED
        try:
            session_batch_status[curr_session_basedir] = SessionBatchProgress.RUNNING
            curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, curr_session_basedir, force_reload=False)
            active_identifying_session_ctx, active_session_figures_out_path, active_out_figures_list = batch_programmatic_figures(curr_active_pipeline)
            batch_extended_programmatic_figures(curr_active_pipeline)
            session_batch_status[curr_session_basedir] = SessionBatchProgress.COMPLETED
            print(f'completed processing for {curr_session_basedir}: {active_identifying_session_ctx}')
        except Exception as e:
            print(f'ERROR processing {curr_session_basedir} with error {e}')
            session_batch_status[curr_session_basedir] = SessionBatchProgress.ABORTED
            # raise e
    else:
        print(f'\t already completed')

print(f'session_batch_status: {session_batch_status}')
print('!!! done running batch !!!')

# 🟢 Single basedir (non-batch) testing:

In [4]:
basedir = local_session_paths_list[2] # NOT 3
print(f'basedir: {str(basedir)}')

# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, skip_save=True, force_reload=False)
## SAVE AFTERWARDS!
# curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, skip_save=False, force_reload=False, active_pickle_filename='loadedSessPickle - full-good.pkl')

basedir: W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43
Loading loaded session pickle file results : W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\loadedSessPickle.pkl... done.
Loading pickled pipeline success: W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\loadedSessPickle.pkl.
property already present in pickled version. No need to save.
is_common_filter_name: [ True  True  True]
is_novel_filter_name: [False False False]
common_filter_names: ['maze1' 'maze2' 'maze']
changed_filters_names_list: []
	 TODO: this will prevent recomputation even when the blacklist/whitelist or computation function definitions change. Rework so that this is smarter.
updating computation_results...
done.
	 TODO: this will prevent recomputation even when the blacklist/whitelist or computation function definitions change. Rework so that this is smarter.
updating computation_results...
done.
	 TODO: this will prevent recomputation even when the blacklist/whitelist or computation function definitions change. Rework so that t

In [ ]:
active_identifying_session_ctx, curr_session_figures_out_path, active_out_figures_list = batch_programmatic_figures(curr_active_pipeline)

In [ ]:
batch_extended_programmatic_figures(curr_active_pipeline)

## 🗨️ 2022-10-26 - Jonathan Firing Rate Analyses

In [5]:
# ==================================================================================================================== #
# Perform missing global computations                                                                                  #
# ==================================================================================================================== #
curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_jonathan_replay_firing_rate_analyses', '_perform_short_long_pf_overlap_analyses'], fail_on_exception=True, debug_print=True) # 

## Get global 'jonathan_firing_rate_analysis' results:
curr_jonathan_firing_rate_analysis = curr_active_pipeline.global_computation_results.computed_data['jonathan_firing_rate_analysis']
neuron_replay_stats_df, rdf, aclu_to_idx, irdf = curr_jonathan_firing_rate_analysis['neuron_replay_stats_df'], curr_jonathan_firing_rate_analysis['rdf']['rdf'], curr_jonathan_firing_rate_analysis['rdf']['aclu_to_idx'], curr_jonathan_firing_rate_analysis['irdf']['irdf']

from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership
short_only_df = neuron_replay_stats_df[neuron_replay_stats_df.track_membership == SplitPartitionMembership.RIGHT_ONLY]
short_only_aclus = short_only_df.index.values.tolist()
long_only_df = neuron_replay_stats_df[neuron_replay_stats_df.track_membership == SplitPartitionMembership.LEFT_ONLY]
long_only_aclus = long_only_df.index.values.tolist()
shared_df = neuron_replay_stats_df[neuron_replay_stats_df.track_membership == SplitPartitionMembership.SHARED]
shared_aclus = shared_df.index.values.tolist()
print(f'shared_aclus: {shared_aclus}')
print(f'long_only_aclus: {long_only_aclus}')
print(f'short_only_aclus: {short_only_aclus}')

active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

global_computation_results is None. Building initial global_computation_results...
Performing _execute_computation_functions(...) with 2 registered_computation_functions...
include_whitelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze
Time window 382 has no spikes.
Time window 396 has no spikes.
include_whitelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze
_execute_computation_functions(...): 
	accumulated_errors: None
shared_aclus: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104]
long_only_aclus: [45]
short_only_aclus: [62]


In [6]:
short_long_pf_overlap_analyses = curr_active_pipeline.global_computation_results.computed_data.short_long_pf_overlap_analyses
conv_overlap_dict = short_long_pf_overlap_analyses['conv_overlap_dict']
conv_overlap_scalars_df = short_long_pf_overlap_analyses['conv_overlap_scalars_df']
prod_overlap_dict = short_long_pf_overlap_analyses['product_overlap_dict']
relative_entropy_overlap_dict = short_long_pf_overlap_analyses['relative_entropy_overlap_dict']
relative_entropy_overlap_scalars_df = short_long_pf_overlap_analyses['relative_entropy_overlap_scalars_df']

In [ ]:
relative_entropy_overlap_scalars_df

In [ ]:
relative_entropy_overlap_dict

In [ ]:
short_xbins = short_results.pf1D.xbin_centers # .shape # (40,)
# short_curves = short_results.pf1D.ratemap.tuning_curves # .shape # (64, 40)
short_curves = short_results.pf1D.ratemap.normalized_tuning_curves # .shape # (64, 40)

long_xbins = long_results.pf1D.xbin_centers # .shape # (63,)
# long_curves = long_results.pf1D.ratemap.tuning_curves # .shape # (64, 63)
long_curves = long_results.pf1D.ratemap.normalized_tuning_curves # .shape # (64, 63)

pf_overlap_conv_results = []
# for i, a_pair in enumerate(pf_neurons_diff.shared.pairs):
i = 5
curr_aclu = pf_neurons_diff.either[i]
a_pair = pf_neurons_diff.shared.pairs[i]
long_idx, short_idx = a_pair
if long_idx is None or short_idx is None:
    # missing entry, answer is zero
    overlap_results_dict = None
else:        
    curr_cell_valid_subset = list(conv_overlap_dict.values())[i]['valid_subset']
    # curr_cell_valid_subset = list(conv_overlap_dict.values())[i]['full']
    x, convolved_result, normalized_convolved_result, area = curr_cell_valid_subset['x'], curr_cell_valid_subset['convolved_result'], curr_cell_valid_subset['normalized_convolved_result'], curr_cell_valid_subset['area']
    
    curr_cell_prod_overlap = list(prod_overlap_dict.values())[i] 
    prod_x, prod_overlap_dot_product, prod_normalized_overlap_dot_product, prod_area, prod_peak_max, prod_extrapolated_short_curve = curr_cell_prod_overlap['x'], curr_cell_prod_overlap['overlap_dot_product'], curr_cell_prod_overlap['normalized_overlap_dot_product'], curr_cell_prod_overlap['area'], curr_cell_prod_overlap['peak_max'], curr_cell_prod_overlap['extrapolated_short_curve'] # Extract variables from the `curr_cell_prod_overlap` dictionary to the local workspace

    long_curve = long_curves[long_idx]
    short_curve = short_curves[short_idx]
        
    overlap_result = {'conv': normalized_convolved_result, 'dot_product': prod_overlap_dot_product}
    overlap_results_dict = _test_plot_conv(long_xbins, long_curve, short_xbins, short_curve, x, overlap_result)
    overlap_results_dict.figures.suptitle(f'aclu: {curr_aclu} - overlap_area: {area}')
pf_overlap_conv_results.append(overlap_results_dict)


pf_overlap_conv_results

In [ ]:
conv_overlap_scalars_df

In [ ]:
conv_overlap_scalars_df

## 🟢 2022-11-21 - 1D Ratemaps Before and After Track change (Long vs. Short track)
Working metrics for comparing overlaps of 1D placefields before and after track change

In [17]:
%pdb off
%matplotlib qt
import matplotlib as mpl
import matplotlib.pyplot as plt
from pyphocorehelpers.plotting.figure_management import PhoActiveFigureManager2D
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons, _compare_computation_results
from neuropy.utils.colors_util import get_neuron_colors
from neuropy.core.neuron_identities import PlotStringBrevityModeEnum

from neuropy.analyses.placefields import PfND # for re-binning pf1D

from neuropy.plotting.figure import Fig
from neuropy.plotting.ratemaps import plot_ratemap_1D

from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import build_neurons_color_map

include_whitelist = curr_active_pipeline.active_completed_computation_result_names # ['maze', 'sprinkle']
long_epoch_name = include_whitelist[0] # 'maze1_PYR'
short_epoch_name = include_whitelist[1] # 'maze2_PYR'
if len(include_whitelist) > 2:
    global_epoch_name = include_whitelist[-1] # 'maze_PYR'
else:
    print(f'WARNING: no global_epoch detected.')
    global_epoch_name = '' # None
long_results = curr_active_pipeline.computation_results[long_epoch_name]['computed_data']
short_results = curr_active_pipeline.computation_results[short_epoch_name]['computed_data']
global_results = curr_active_pipeline.computation_results[global_epoch_name]['computed_data']

long_pf1D = long_results.pf1D
short_pf1D = short_results.pf1D
global_pf1D = global_results.pf1D

## Allow overriding PfND's bins:
# TODO: 2022-12-09 - We want to be able to have both long/short track placefields have the same bins. 
if (len(short_pf1D.xbin) < len(long_pf1D.xbin)):
    print(f'short_pf1D will be re-binned to match long_pf1D...')
    bak_short_pf1D = deepcopy(short_pf1D) # Backup the original first
    xbin, ybin, bin_info, grid_bin = long_pf1D.xbin, long_pf1D.ybin, long_pf1D.bin_info, long_pf1D.config.grid_bin
    ## Apply to the short dataframe:
    short_pf1D.xbin, short_pf1D.ybin, short_pf1D.bin_info, short_pf1D.config.grid_bin = xbin, ybin, bin_info, grid_bin
    ## Updates (replacing) the 'binned_x' (and if 2D 'binned_y') columns to the position dataframe:
    short_pf1D._filtered_pos_df, _, _, _ = PfND.build_position_df_discretized_binned_positions(short_pf1D._filtered_pos_df, short_pf1D.config, xbin_values=short_pf1D.xbin, ybin_values=short_pf1D.ybin, debug_print=False) # Finishes setup
    short_pf1D.compute() # does compute
    print(f'done.') ## Successfully re-bins pf1D:

## Get the normalized_tuning_curves only for the shared aclus (that are common across (long/short/global):
long_is_included = np.isin(long_pf1D.ratemap.neuron_ids, shared_aclus)  #.shape # (104, 63)
long_incl_aclus = np.array(long_pf1D.ratemap.neuron_ids)[long_is_included] #.shape # (98,)
long_incl_curves = long_pf1D.ratemap.normalized_tuning_curves[long_is_included]  #.shape # (98, 63)
assert long_incl_aclus.shape[0] == long_incl_curves.shape[0] # (98,) == (98, 63)

short_is_included = np.isin(short_pf1D.ratemap.neuron_ids, shared_aclus)
short_incl_aclus = np.array(short_pf1D.ratemap.neuron_ids)[short_is_included] #.shape (98,)
short_incl_curves = short_pf1D.ratemap.normalized_tuning_curves[short_is_included]  #.shape # (98, 40)
assert short_incl_aclus.shape[0] == short_incl_curves.shape[0] # (98,) == (98, 63)
# assert short_incl_curves.shape[1] == long_incl_curves.shape[1] # short and long should have the same bins

global_is_included = np.isin(global_pf1D.ratemap.neuron_ids, shared_aclus)
global_incl_aclus = np.array(global_pf1D.ratemap.neuron_ids)[global_is_included] #.shape (98,)
global_incl_curves = global_pf1D.ratemap.normalized_tuning_curves[global_is_included]  #.shape # (98, 63)
assert global_incl_aclus.shape[0] == global_incl_curves.shape[0] # (98,) == (98, 63)
assert global_incl_curves.shape[1] == long_incl_curves.shape[1] # global and long should have the same bins

assert np.alltrue(np.isin(long_incl_aclus, short_incl_aclus))
assert np.alltrue(np.isin(long_incl_aclus, global_incl_aclus))


## get shared neuron info:
pf_neurons_diff = _compare_computation_results(long_results.pf1D.ratemap.neuron_ids, short_results.pf1D.ratemap.neuron_ids)
curr_any_context_neurons = pf_neurons_diff.either
long_only_aclus = pf_neurons_diff.lhs_only
short_only_aclus = pf_neurons_diff.rhs_only
print(f'long_only_aclus: {long_only_aclus}\nshort_only_aclus: {short_only_aclus}')
n_neurons = pf_neurons_diff.shared.n_neurons
shared_fragile_neuron_IDXs = pf_neurons_diff.shared.shared_fragile_neuron_IDXs

neurons_colors_array = build_neurons_color_map(n_neurons, sortby=shared_fragile_neuron_IDXs, cmap=None)

sort_idx = shared_fragile_neuron_IDXs.copy()    
print(f'n_neurons: {n_neurons}')
print(f'shared_fragile_neuron_IDXs: {shared_fragile_neuron_IDXs}.\t np.shape: {np.shape(shared_fragile_neuron_IDXs)}')
print(f'curr_any_context_neurons: {curr_any_context_neurons}.\t np.shape: {np.shape(curr_any_context_neurons)}')

Automatic pdb calling has been turned OFF
long_only_aclus: [45]
short_only_aclus: [62]
n_neurons: 103
shared_fragile_neuron_IDXs: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102].	 np.shape: (103,)
curr_any_context_neurons: [  2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 

In [23]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.MultiContextComputationFunctions import compute_relative_entropy_divergence_overlap

relative_entropy_overlap_dict, relative_entropy_overlap_scalars_df = compute_relative_entropy_divergence_overlap(long_results, short_results, debug_print=False)
relative_entropy_overlap_scalars_df

In [27]:
aclu_keys = list(relative_entropy_overlap_dict.keys())

len(aclu_keys) # 103

In [37]:
aclu_keys = [k for k,v in relative_entropy_overlap_dict.items() if v is not None]
# len(aclu_keys) # 101
short_long_rel_entr_curves = np.vstack([v['short_long_rel_entr_curve'] for k,v in relative_entropy_overlap_dict.items() if v is not None])
short_long_rel_entr_curves # .shape # (101, 63)

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -2.58258502e-06, -1.29260471e-05, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.75731319e-06, -9.82737300e-06, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-1.06353961e-05, -7.46884887e-05, -3.54054052e-04, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -2.97476808e-06, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-9.01277068e-07, -4.76866205e-06, -1.89809518e-05, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [46]:
## Time Dependent 1D Placefields:
from neuropy.analyses.time_dependent_placefields import PfND_TimeDependent

computation_result = curr_active_pipeline.computation_results[global_epoch_name]
active_session, pf_computation_config = computation_result.sess, computation_result.computation_config.pf_params
active_session_spikes_df, active_pos, computation_config, active_epoch_placefields1D, active_epoch_placefields2D, included_epochs, should_force_recompute_placefields = active_session.spikes_df, active_session.position, pf_computation_config, None, None, pf_computation_config.computation_epochs, True

In [55]:
active_pf_1D = global_pf1D
active_pf_1D_dt = PfND_TimeDependent(deepcopy(active_session_spikes_df), deepcopy(active_pos.linear_pos_obj), epochs=included_epochs,
                                    speed_thresh=computation_config.speed_thresh, frate_thresh=computation_config.frate_thresh,
                                    grid_bin=computation_config.grid_bin, grid_bin_bounds=computation_config.grid_bin_bounds, smooth=computation_config.smooth)



In [51]:
active_pf_1D_dt

In [56]:
## Update the time-dependent pf:
active_pf_1D_dt.reset()

## Find the neuron_IDs that are included in the active_pf_1D for filtering the active_pf_1D_dt's results:
is_pf_1D_included_neuron = np.isin(active_pf_1D_dt.included_neuron_IDs, active_pf_1D.included_neuron_IDs)
pf_1D_included_neuron_indx = active_pf_1D_dt._included_thresh_neurons_indx[is_pf_1D_included_neuron]
active_pf_1D_dt._included_thresh_neurons_indx = pf_1D_included_neuron_indx
active_pf_1D_dt._peak_frate_filter_function = lambda list_: [list_[_] for _ in active_pf_1D_dt._included_thresh_neurons_indx]

earliest_pos_t = active_pf_1D_dt.all_time_filtered_pos_df['t'].values[0]
print(f'earliest_pos_t: {earliest_pos_t}')
# active_pf_1D_dt.update(t=earliest_pos_t)
# active_pf_1D_dt.step(num_seconds_to_advance=6000.0)

last_pos_t = active_pf_1D_dt.all_time_filtered_pos_df['t'].values[-1]
print(f'last_pos_t: {last_pos_t}')
# active_pf_1D_dt.update(t=last_pos_t)

In [57]:
active_pf_1D_dt.xbin

In [58]:
active_pf_1D_dt.ybin

In [59]:
active_pf_1D_dt.config

<PlacefieldComputationParameters: {'speed_thresh': 10.0, 'grid_bin': (3.8054171165052444, 1.4477079927649104), 'grid_bin_bounds': (None, None), 'smooth': (2.0, 2.0), 'frate_thresh': 0.2};>

In [66]:
# active_pf_1D_dt.update(t=earliest_pos_t+(60.0 * 5.0))
active_pf_1D_dt.update(t=3000000.0)
print(f'post-update time: {active_pf_1D_dt.last_t}')

# earliest_pos_t: 22.26785500004189
# last_pos_t: 1739.1316560000414
# post-update time: 322.2678550000419

ndim: 1, position_column_names: ('x',), binned_column_names: ('binned_x',)
ERROR: 2D!!!
ndim: 2, position_column_names: ('x', 'y'), binned_column_names: ('binned_x', 'binned_y')
np.isscalar(bin_size): 1.4477079927649104


ValueError: operands could not be broadcast together with shapes (103,63,61) (63,) 

In [51]:
active_pf_2D_dt.reset()
active_pf_2D_dt.update(t=45.0, start_relative_t=True)

In [51]:
from pyphoplacecellanalysis.GUI.Qt.Menus.SpecificMenus.CreateLinkedWidget_MenuProvider import CreateNewTimeSynchronizedPlotterCommand

curr_window.ui.menus.global_window_menus.create_linked_widget.actions_dict['actionCreateNewTimeSynchronizedOccupancyPlotter'] = CreateNewTimeSynchronizedPlotterCommand(spike_raster_window, active_pf_2D_dt, plotter_type='occupancy', display_output=display_output)

<PfND_TimeDependent: {'config': <PlacefieldComputationParameters: {'speed_thresh': 10.0, 'grid_bin': (3.8054171165052444, 1.4477079927649104), 'grid_bin_bounds': (None, None), 'smooth': (2.0, 2.0), 'frate_thresh': 0.2};>, 'position_srate': 29.96972244523928, 'ndim': 1, '_included_thresh_neurons_indx': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102]), '_peak_frate_filter_function': <function PfND_TimeDependent.__init__.<locals>.<lambda> at 0x000001E146189820>, '_filtered_po

In [ ]:
plt.plot(short_long_rel_entr_curves.T)

In [18]:
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

long_single_cell_pfmap_processing_fn = None
short_single_cell_pfmap_processing_fn = None

# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: 0.5 * pfmap # flip over the y-axis
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: -0.5 * pfmap # flip over the y-axis

# pad = 1
# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap) + (0.5*pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (-0.5 * pfmap * pad) + (0.5*pad) # flip over the y-axis, shift the baseline down by half

# pad = 1
# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) + (0.5*pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) + (0.5*pad) # flip over the y-axis, shift the baseline down by half
# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) # flip over the y-axis, shift the baseline down by half


# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (1.0 * pfmap * pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (-1.0 * pfmap * pad) + (1.0*pad) # this does not work and results in short being fully filled. I think this is because the fill_between gets reversed since everything is below baseline


out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=True, debug_print=False, fignum='Short v Long pf1D Comparison',
                                   long_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
                                   short_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
                                  )
ax = out.axes[0]

In [19]:
## Overlap Scalar Comparisons: plots a comparison of a specific type of scalar values for all cells
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.MultiContextComparingDisplayFunctions import PlacefieldOverlapMetricMode

active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

# overlap_metric_mode = PlacefieldOverlapMetricMode.POLY
# overlap_metric_mode = PlacefieldOverlapMetricMode.PRODUCT
# overlap_metric_mode = PlacefieldOverlapMetricMode.CONVOLUTION
overlap_metric_mode = PlacefieldOverlapMetricMode.REL_ENTROPY

out = curr_active_pipeline.display('_display_short_long_pf1D_scalar_overlap_comparison', active_identifying_session_ctx, overlap_metric_mode=overlap_metric_mode, variant_name='_area')

## 2022-12-09 - Pho Surprise/KL-Divergence Metrics

In [ ]:
short_pf1D.ratemap.never_visited_occupancy_mask

In [ ]:
short_pf1D.nan_never_visited_occupancy

In [ ]:
short_pf1D.xbin

In [ ]:
long_pf1D.xbin

In [ ]:
short_pf1D.bin_info # {'mode': 'bin_size', 'xstep': 3.793023081021702, 'xnum_bins': 41}

In [ ]:
short_pf1D.config

In [ ]:
long_pf1D.bin_info

In [ ]:
long_pf1D.config

In [12]:
short_pf1D.bin_info

{'mode': 'bin_size', 'xstep': 3.8054171165052444, 'xnum_bins': 64}

In [13]:
short_pf1D.xbin.shape

(64,)

In [14]:
short_pf1D.config

<PlacefieldComputationParameters: {'speed_thresh': 10.0, 'grid_bin': (3.8054171165052444, 1.4477079927649104), 'grid_bin_bounds': (None, None), 'smooth': (2.0, 2.0), 'frate_thresh': 0.2};>

In [ ]:
long_incl_curves

In [ ]:
short_incl_curves

In [ ]:
# %matplotlib qt

# ==================================================================================================================== #
# Output Figures to File                                                                                               #
# ==================================================================================================================== #
## PDF Output
# %matplotlib qtagg
import matplotlib
# configure backend here
# matplotlib.use('Qt5Agg')
# backend_qt5agg
matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

n_max_page_rows = 10
_batch_plot_kwargs_list = _build_batch_plot_kwargs(long_only_aclus, short_only_aclus, shared_aclus, active_identifying_session_ctx, n_max_page_rows=n_max_page_rows)
active_out_figures_list = _perform_batch_plot(curr_active_pipeline, _batch_plot_kwargs_list, figures_parent_out_path=None, write_pdf=False, write_png=True, progress_print=True, debug_print=False)

## ❌🆖 BROKEN Individual Plotting Outputs:

### Common Config

In [ ]:
## MATPLOTLIB Imports:
import matplotlib
# configure backend here
matplotlib.use('Qt5Agg')
# backend_qt5agg
# matplotlib.use('AGG') # non-interactive backend
## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.backends import backend_pdf

from neuropy.utils.matplotlib_helpers import enumTuningMap2DPlotVariables # for getting the variant name from the dict
_bak_rcParams = mpl.rcParams.copy()
mpl.rcParams['toolbar'] = 'None' # disable toolbars

from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, build_pdf_metadata_from_display_context, programmatic_display_to_PDF

# from pyphocorehelpers.plotting.figure_management import PhoActiveFigureManager2D, capture_new_figures_decorator
# fig_man = PhoActiveFigureManager2D(name=f'fig_man') # Initialize a new figure manager

active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

In [ ]:
def process_session_plots(curr_active_pipeline, active_config_name, debug_print=False):
    """ Unwrap single config 
    UNUSED AND UNTESTED
    """
    print(f'active_config_name: {active_config_name}')

    ## Add the filter to the active context
    # active_identifying_filtered_session_ctx = active_identifying_session_ctx.adding_context('filter', filter_name=active_config_name) # 'bapun_RatN_Day4_2019-10-15_11-30-06_maze'
    active_identifying_filtered_session_ctx = curr_active_pipeline.filtered_contexts[active_config_name] # 'bapun_RatN_Day4_2019-10-15_11-30-06_maze'

    # Get relevant variables:
    # curr_active_pipeline is set above, and usable here
    sess: DataSession = curr_active_pipeline.filtered_sessions[active_config_name]

    active_computation_results = curr_active_pipeline.computation_results[active_config_name]
    active_computed_data = curr_active_pipeline.computation_results[active_config_name].computed_data
    active_computation_config = curr_active_pipeline.computation_results[active_config_name].computation_config
    active_computation_errors = curr_active_pipeline.computation_results[active_config_name].accumulated_errors
    print(f'active_computed_data.keys(): {list(active_computed_data.keys())}')
    print(f'active_computation_errors: {active_computation_errors}')
    active_pf_1D = curr_active_pipeline.computation_results[active_config_name].computed_data['pf1D']
    active_pf_2D = curr_active_pipeline.computation_results[active_config_name].computed_data['pf2D']
    active_pf_1D_dt = curr_active_pipeline.computation_results[active_config_name].computed_data.get('pf1D_dt', None)
    active_pf_2D_dt = curr_active_pipeline.computation_results[active_config_name].computed_data.get('pf2D_dt', None)
    active_firing_rate_trends = curr_active_pipeline.computation_results[active_config_name].computed_data.get('firing_rate_trends', None)
    active_one_step_decoder = curr_active_pipeline.computation_results[active_config_name].computed_data.get('pf2D_Decoder', None)
    active_two_step_decoder = curr_active_pipeline.computation_results[active_config_name].computed_data.get('pf2D_TwoStepDecoder', None)
    active_extended_stats = curr_active_pipeline.computation_results[active_config_name].computed_data.get('extended_stats', None)
    active_eloy_analysis = curr_active_pipeline.computation_results[active_config_name].computed_data.get('EloyAnalysis', None)
    active_simpler_pf_densities_analysis = curr_active_pipeline.computation_results[active_config_name].computed_data.get('SimplerNeuronMeetingThresholdFiringAnalysis', None)
    active_ratemap_peaks_analysis = curr_active_pipeline.computation_results[active_config_name].computed_data.get('RatemapPeaksAnalysis', None)
    active_peak_prominence_2d_results = curr_active_pipeline.computation_results[active_config_name].computed_data.get('RatemapPeaksAnalysis', {}).get('PeakProminence2D', None)
    active_measured_positions = curr_active_pipeline.computation_results[active_config_name].sess.position.to_dataframe()
    curr_spikes_df = sess.spikes_df

    curr_active_config = curr_active_pipeline.active_configs[active_config_name]
    curr_active_display_config = curr_active_config.plotting_config

    active_display_output = curr_active_pipeline.display_output[active_identifying_filtered_session_ctx]
    print(f'active_display_output: {active_display_output}')

    # Create `master_dock_win` - centralized plot output window to collect individual figures/controls in (2022-08-18)
    display_output = active_display_output | curr_active_pipeline.display('_display_context_nested_docks', active_identifying_session_ctx, enable_gui=False, debug_print=True) # returns {'master_dock_win': master_dock_win, 'app': app, 'out_items': out_items}
    master_dock_win = display_output['master_dock_win']
    app = display_output['app']
    out_items = display_output['out_items']

    def _get_curr_figure_format_config():
        """ Aims to fetch the current figure_format_config and context from the figure_format_config widget:    
        Implicitly captures: `out_items`, `active_config_name`, `active_identifying_filtered_session_ctx` 
        """
        ## Get the figure_format_config from the figure_format_config widget:
        # Fetch the context from the GUI:
        _curr_gui_session_ctx, _curr_gui_out_display_items = out_items[active_config_name]
        _curr_gui_figure_format_config_widget = _curr_gui_out_display_items[active_identifying_filtered_session_ctx.adding_context('display_fn', display_fn_name='figure_format_config_widget')] # [0] is seemingly not needed to unpack the tuple
        if _curr_gui_figure_format_config_widget is not None:
            # has GUI for config
            figure_format_config = _curr_gui_figure_format_config_widget.figure_format_config
        else:
            # has non-GUI provider of figure_format_config
            figure_format_config = _curr_gui_figure_format_config_widget.figure_format_config

        if debug_print:
            print(f'recovered gui figure_format_config: {figure_format_config}')

        return figure_format_config

    figure_format_config = _get_curr_figure_format_config()

    ## PDF Output, NOTE this is single plot stuff: uses active_config_name
    from matplotlib.backends import backend_pdf
    from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_pdf_export_metadata

    filter_name = active_config_name
    _build_pdf_pages_output_info, programmatic_display_function_testing_output_parent_path = build_pdf_export_metadata(curr_active_pipeline.sess.get_description(), filter_name=filter_name)
    print(f'Figure Output path: {str(programmatic_display_function_testing_output_parent_path)}')
    
    
    ## Test getting figure save paths:
    _test_fig_path = curr_active_config.plotting_config.get_figure_save_path('test')
    print(f'_test_fig_path: {_test_fig_path}\n\t exists? {_test_fig_path.exists()}')

    return active_identifying_filtered_session_ctx, programmatic_display_function_testing_output_parent_path
    

# active_config_name = 'maze1'
# active_config_name = 'maze2'
# active_config_name = 'maze'
# active_config_name = 'sprinkle'

# active_config_name = 'maze_PYR'

# active_config_name = 'maze1_rippleOnly'
# active_config_name = 'maze2_rippleOnly'

# active_config_name = curr_active_pipeline.active_config_names[0] # get the first name by default
active_config_name = curr_active_pipeline.active_config_names[-1] # get the last name

active_identifying_filtered_session_ctx, programmatic_display_function_testing_output_parent_path = process_session_plots(curr_active_pipeline, active_config_name)

### Single (Session, Filter) Context Plotting:

#### Utility:

In [ ]:
# Reload display functions:
curr_active_pipeline.reload_default_display_functions()
curr_active_pipeline.registered_display_function_names # ['_display_1d_placefield_validations', '_display_2d_placefield_result_plot_ratemaps_2D', '_display_2d_placefield_result_plot_raw', '_display_normal', '_display_placemaps_pyqtplot_2D', '_display_decoder_result', '_display_plot_most_likely_position_comparisons', '_display_two_step_decoder_prediction_error_2D', '_display_two_step_decoder_prediction_error_animated_2D', '_display_spike_rasters_pyqtplot_2D', '_display_spike_rasters_pyqtplot_3D', '_display_spike_rasters_pyqtplot_3D_with_2D_controls', '_display_spike_rasters_vedo_3D', '_display_spike_rasters_vedo_3D_with_2D_controls', '_display_spike_rasters_window', '_display_speed_vs_PFoverlapDensity_plots', '_display_3d_image_plotter', '_display_3d_interactive_custom_data_explorer', '_display_3d_interactive_spike_and_behavior_browser', '_display_3d_interactive_tuning_curves_plotter']
print(curr_active_pipeline.registered_display_function_names)

In [ ]:
%matplotlib --list 
# Available matplotlib backends: ['tk', 'gtk', 'gtk3', 'gtk4', 'wx', 'qt4', 'qt5', 'qt6', 'qt', 'osx', 'nbagg', 'notebook', 'agg', 'svg', 'pdf', 'ps', 'inline', 'ipympl', 'widget']

In [ ]:
%matplotlib qt
## NOTE THAT ONCE THIS IS SET TO qt, it cannot be undone!

### Systematic Display Function Testing

#### Matplotlib-based plots:

In [ ]:
import matplotlib
# matplotlib.use('AGG') # non-interactive backend
# %matplotlib -l

matplotlib.use('Qt5Agg') # non-interactive backend
## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

curr_active_pipeline.filtered_session_names # ['maze', 'sprinkle']
active_config_name = 'maze'

active_display_to_pdf_fn = programmatic_display_to_PDF

In [ ]:
%%capture
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_1d_placefield_validations') # 🟢✅ Now seems to be working and saving to PDF!! Still using matplotlib.use('Qt5Agg') mode and plots still appear. Moderate visual improvements can still be made (titles overlap and stuff). Works with %%capture

# active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_1d_placefield_validations', filter_name=active_config_name) # 🟢✅ Now seems to be working and saving to PDF!! Still using matplotlib.use('Qt5Agg') mode and plots still appear. Moderate visual improvements can still be made (titles overlap and stuff). Works with %%capture

In [ ]:
# %%capture
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_2d_placefield_result_plot_raw', debug_print=False) # 🔇🆖❌ IndexError: index 80 is out of bounds for GridSpec with size 80

In [ ]:
# %%capture
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_1d_placefields', debug_print=False) # 🟢✅ Now seems to be working and saving to PDF!! Still using matplotlib.use('Qt5Agg') mode and plots still appear.

In [ ]:
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_1d_placefields', debug_print=True)

In [ ]:
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_normal', debug_print=True) # 🐞❌ TypeError: unhashable type: 'list'

In [ ]:
# %%capture
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_2d_placefield_result_plot_ratemaps_2D') #  🟢✅ Now seems to be working and saving to PDF!! Still using matplotlib.use('Qt5Agg') mode and plots still appear.

In [ ]:
%%capture
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_normal', filter_name=active_config_name) # 🐞❌ TypeError: unhashable type: 'list'

### 🐞👁️‍🗨️🔜 TODO: FINISH THIS UP AND FIGURE OUT WHATEVER THE HELL I'M DOING HERE

In [ ]:
curr_display_function_name = '_display_2d_placefield_result_plot_ratemaps_2D'
built_pdf_metadata, curr_pdf_save_path = _build_pdf_pages_output_info(curr_display_function_name)
out_fig_list = []
active_identifying_display_ctx = active_identifying_filtered_session_ctx.adding_context('display_fn', display_fn_name=curr_display_function_name)
figure_format_config = _get_curr_figure_format_config() # Fetch the context from the GUI
figure_format_config['enable_saving_to_disk'] = False # don't use the in-built figure export/saving to disk functionality as we want to wrap the output figure with the Pdf saving, not write to a .png
with backend_pdf.PdfPages(curr_pdf_save_path, keep_empty=False, metadata=built_pdf_metadata) as pdf:
    ## TypeError: neuropy.utils.debug_helpers.safely_accepts_kwargs.<locals>._safe_kwargs_fn() got multiple values for keyword argument 'computation_config'
    for filter_name in curr_active_pipeline.filtered_session_names:
        print(f'filter_name: {filter_name}')
        active_identifying_ctx = active_identifying_display_ctx.adding_context('plot_variable', variable_name=enumTuningMap2DPlotVariables.SPIKES_MAPS)
        active_identifying_ctx_string = active_identifying_ctx.get_description(separator='|') # Get final discription string
        out_fig_list.extend(curr_active_pipeline.display(curr_display_function_name, filter_name, plot_variable=enumTuningMap2DPlotVariables.SPIKES_MAPS, fignum=active_identifying_ctx_string, **figure_format_config)) # works!
        active_identifying_ctx = active_identifying_display_ctx.adding_context('plot_variable', variable_name=enumTuningMap2DPlotVariables.TUNING_MAPS)
        active_identifying_ctx_string = active_identifying_ctx.get_description(separator='|') # Get final discription string
        out_fig_list.extend(curr_active_pipeline.display(curr_display_function_name, filter_name, plot_variable=enumTuningMap2DPlotVariables.TUNING_MAPS, fignum=active_identifying_ctx_string, **figure_format_config))
        for a_fig in out_fig_list:
            pdf.savefig(a_fig, transparent=True)
            
# 🐞🔇🆖❌ NameError: name '_build_pdf_pages_output_info' is not defined

In [ ]:
%%capture
curr_display_function_name = '_display_decoder_result'
built_pdf_metadata, curr_pdf_save_path = _build_pdf_pages_output_info(curr_display_function_name)
with backend_pdf.PdfPages(curr_pdf_save_path, keep_empty=False, metadata=built_pdf_metadata) as pdf:
    plots = curr_active_pipeline.display(curr_display_function_name, filter_name)
    print(plots)
    # pdf.savefig(a_fig)
    
    
# 🐞🔇🆖❌ NameError: name '_build_pdf_pages_output_info' is not defined

#### PyQtGraph-based plots:

#### PyQtGraph-based Pf2D Viewers:

In [ ]:
# 🟢✅ Nearly Completely Working - Needs subplot labels changed to match standardized matplotlib version, needs color scheme set consistently to matplotlib version, needs colorbars removed
from pyphoplacecellanalysis.GUI.PyQtPlot.BinnedImageRenderingWindow import BasicBinnedImageRenderingWindow, add_bin_ticks, build_binned_imageItem
from neuropy.utils.matplotlib_helpers import _build_variable_max_value_label, enumTuningMap2DPlotMode, enumTuningMap2DPlotVariables, _determine_best_placefield_2D_layout, _scale_current_placefield_to_acceptable_range
from pyphoplacecellanalysis.Pho2D.PyQtPlots.plot_placefields import display_all_pf_2D_pyqtgraph_binned_image_rendering

# NOTE FILTER SPECIFIC: active_config_name and active_pf_2D depend on active_config_name

## Get the figure_format_config from the figure_format_config widget:
active_identifying_display_ctx = active_identifying_filtered_session_ctx.adding_context('display_fn', display_fn_name='display_all_pf_2D_pyqtgraph_binned_image_rendering')
figure_format_config = _get_curr_figure_format_config() # Fetch the context from the GUI
out_all_pf_2D_pyqtgraph_binned_image_fig = display_all_pf_2D_pyqtgraph_binned_image_rendering(active_pf_2D, figure_format_config)

In [ ]:
out_all_pf_2D_pyqtgraph_binned_image_fig.setWindowTitle(f'{active_identifying_display_ctx.get_description()}')

In [ ]:
images = active_one_step_decoder.ratemap.normalized_tuning_curves
images.shape # (66, 41, 63)

In [ ]:
# 🟢🚧🟨 Almost Working - Needs subplot labels changed from Cell[i] to the appropriate standardized titles. Needs other minor refinements.
# 🚧 pyqtplot_plot_image_array needs major improvements to achieve feature pairity with display_all_pf_2D_pyqtgraph_binned_image_rendering, so probably just use display_all_pf_2D_pyqtgraph_binned_image_rendering.  
from pyphoplacecellanalysis.Pho2D.PyQtPlots.plot_placefields import pyqtplot_plot_image_array

# Get the decoders from the computation result:       
# Get flat list of images:
images = active_one_step_decoder.ratemap.normalized_tuning_curves # (43, 63, 63)
occupancy = active_one_step_decoder.ratemap.occupancy

active_identifying_display_ctx = active_identifying_filtered_session_ctx.adding_context('display_fn', display_fn_name='pyqtplot_plot_image_array')
figure_format_config = _get_curr_figure_format_config() # Fetch the context from the GUI
## Get final discription string:
active_identifying_ctx_string = active_identifying_display_ctx.get_description(separator='|')
print(f'active_identifying_ctx_string: {active_identifying_ctx_string}')

## Build the widget:
app, parent_root_widget, root_render_widget, plot_array, img_item_array, other_components_array = pyqtplot_plot_image_array(active_one_step_decoder.xbin, active_one_step_decoder.ybin, images, occupancy, 
                                                                        app=None, parent_root_widget=None, root_render_widget=None, max_num_columns=8)
parent_root_widget.show()
if master_dock_win is not None:
    # if there's an open master_dock_win, add this widget as a child dock
    master_dock_win.add_display_dock(identifier=active_identifying_ctx_string, widget=parent_root_widget, dockIsClosable=True)

#### Decoder Plots:

In [ ]:
# Must switch back to the interactive backend here for the interactive/animated decoder plots:
matplotlib.use('Qt5Agg')
# backend_qt5agg
import matplotlib.pyplot as plt
# plt.switch_backend('Qt5Agg')

In [ ]:
curr_active_pipeline.display('_display_two_step_decoder_prediction_error_animated_2D', active_config_name, variable_name='p_x_given_n')

In [ ]:
# ## MATPLOTLIB Imports:
# import matplotlib
# # configure backend here
# matplotlib.use('Qt5Agg')
# import matplotlib.pyplot as plt
# import matplotlib as mpl
## This plot looks phenominal, and the slider works!
curr_active_pipeline.display('_display_two_step_decoder_prediction_error_2D', active_config_name, variable_name='p_x_given_n') # NOW: TypeError: _temp_debug_two_step_plots_animated_imshow() missing 1 required positional argument: 'time_binned_position_df'

In [ ]:
curr_active_pipeline.display('_display_two_step_decoder_prediction_error_2D', active_config_name, variable_name='p_x_given_n_and_x_prev')  # this one doesn't work!

In [ ]:
# Get the decoders from the computation result:
# active_one_step_decoder = computation_result.computed_data['pf2D_Decoder']
# active_two_step_decoder = computation_result.computed_data.get('pf2D_TwoStepDecoder', None)
# active_measured_positions = computation_result.sess.position.to_dataframe()

active_one_step_decoder # BayesianPlacemapPositionDecoder
active_two_step_decoder

## SAVE OUT THE RESULTS of the decoder:

In [ ]:
## PDF Output, NOTE this is single plot stuff: uses active_config_name
from matplotlib.backends import backend_pdf
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, build_pdf_metadata_from_display_context, programmatic_display_to_PDF

In [ ]:
## 2022-10-04 Modern Programmatic PDF outputs:
# programmatic_display_to_PDF(curr_active_pipeline, curr_display_function_name='_display_plot_decoded_epoch_slices',  debug_print=False)
programmatic_display_to_PDF(curr_active_pipeline, curr_display_function_name='_display_plot_decoded_epoch_slices', filter_epochs='ripple', decoding_time_bin_size=0.02, debug_test_max_num_slices=128, debug_print=True)

In [ ]:
programmatic_display_to_PDF(curr_active_pipeline, curr_display_function_name='_display_plot_decoded_epoch_slices', filter_epochs='laps', debug_test_max_num_slices=128, debug_print=False)

### 🔜 2022-08-10 👁️‍🗨️ NOW: Plot animal positions on the computed posteriors:
The process of plotting the animal position on the decoder plot needs to be refined. Currently it works by re-implementing 

🔜 NEXT STEP: TODO: Make a "Datasource" like approach perhaps to provide the actual animal position at each point in time?
🐞🔜 BUG TODO: Noticed that for Bapun Day5 data, it looks like the current position point is being plotted incorrectly (it doesn't even move across the space much)

In [ ]:
## Get the current positions at each of the time_window_centers:
# active_resampled_measured_positions
time_binned_pos_df = active_computed_data.extended_stats.time_binned_position_df
active_resampled_pos_df = time_binned_pos_df  # 1717 rows × 16 columns
active_resampled_measured_positions = active_resampled_pos_df[['x','y']].to_numpy() # The measured positions resampled (interpolated) at the window centers. 
# np.shape(active_resampled_measured_positions) # (1911, 2)
active_one_step_decoder.active_time_window_centers.shape # (1911,)
print(f'active_one_step_decoder.active_time_window_centers.shape: {active_one_step_decoder.active_time_window_centers.shape}')
# Note this has 2900 rows × 24 columns and active_one_step_decoder.active_time_window_centers.shape is (2892,) for some reason. Shouldn't they be the same?

In [ ]:
active_resampled_pos_df

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

from PendingNotebookCode import _temp_debug_two_step_plots_animated_imshow

# Get the decoders from the computation result:
# active_one_step_decoder = computation_result.computed_data['pf2D_Decoder']
# active_two_step_decoder = computation_result.computed_data.get('pf2D_TwoStepDecoder', None)
# active_measured_positions = computation_result.sess.position.to_dataframe()

def _debug_on_frame_update(new_frame_idx, ax):
    print(f'_debug_on_frame_update(new_frame_idx: {new_frame_idx}, ax: {ax})')
    pass

# active_resampled_pos_df = active_computed_data.extended_stats.time_binned_position_df  # 1717 rows × 16 columns

# Simple plot type 1:
# plotted_variable_name = kwargs.get('variable_name', 'p_x_given_n') # Tries to get the user-provided variable name, otherwise defaults to 'p_x_given_n'
plotted_variable_name = 'p_x_given_n' # Tries to get the user-provided variable name, otherwise defaults to 'p_x_given_n'
_temp_debug_two_step_plots_animated_imshow(active_one_step_decoder, active_two_step_decoder, active_computed_data.extended_stats.time_binned_position_df, variable_name=plotted_variable_name, update_callback_function=_debug_on_frame_update) # Works

In [ ]:
curr_display_function_name = '_display_spike_rasters_pyqtplot_2D'
curr_active_pipeline.display(curr_display_function_name, filter_name, debug_print=False, enable_saving_to_disk=enable_saving_to_disk) 

In [ ]:
## Works, displays my velocity/density result for both 2D and 1D:
# out_plot_1D, out_plot_2D = curr_active_pipeline.display('_display_speed_vs_PFoverlapDensity_plots', active_config_name)
curr_display_function_name = '_display_speed_vs_PFoverlapDensity_plots'
plots = curr_active_pipeline.display(curr_display_function_name, filter_name)
plots

In [ ]:
curr_display_function_name = '_display_placemaps_pyqtplot_2D'
out_plots = curr_active_pipeline.display(curr_display_function_name, filter_name, max_num_columns=8)    
out_plots[1].show()

In [ ]:
# a_plot = plots[0] # PlotWidget 
# a_plot_item = a_plot.plotItem # PlotItem
# a_plot.scene() # GraphicsScene
export_pyqtgraph_plot(plots[0])